In [1]:
from pyconsolida.budget_reader import *
from pyconsolida.budget_reader import _get_valid_costo_rows, _read_raw_budget_sheet
from pathlib import Path
import pandas as pd
import flammkuchen as fl
import numpy as np

## Compute difference

In [14]:
dest_dir = Path("/Users/vigji/Desktop/icop/exported_luigi_sum-fasi-False")

tstamp = "231115_030541"

In [15]:
data_dict = fl.load(dest_dir / f"{tstamp}_python_data.h5")
numerical = ["quantita", "imp. unit."]
budgets_tot, budgets_dec = data_dict["budgets"], data_dict["budgets_dec"]

In [16]:
budgets_tot

,commessa,fase,codice,tipologia,voce,costo u.,u.m.,quantita,imp. unit.,imp.comp.
0,1468,Spese Generali,620,Manodopera,Direttore microtunnel,500.0,g,10.5,51.980198,5250.0
1,1468,Spese Generali,621,Manodopera,Project Manager,350.0,g,104.0,360.39604,36400.0
2,1468,Spese Generali,622,Manodopera,Construction Manager,300.0,g,85.5,253.960396,25650.0
3,1468,Spese Generali,623,Manodopera,ASPP,300.0,g,46.0,136.633663,13800.0
4,1468,Spese Generali,625,Manodopera,Impiegato sede (Bergamasco),200.0,g,18.5,36.633663,3700.0
...,...,...,...,...,...,...,...,...,...,...
16121,1504,Fase2,1106,Materiali di consumo,Gasolio,11.83,l,3996.0,40.390191,47272.68
16122,1504,Fase2,1108,Materiali di consumo,"Materiale vario di consumo (elettrodi, ecc.)",745.0,cad,5.0,3.182673,3725.0
16123,1504,Fase2,1103,Materiali di consumo,Picchi utensili,195.5625,cad,5.0,0.835452,977.8125
16124,1504,Fase2,1116,Materiali di consumo,Tappi per CFA,52.15,cad,64.0,2.851675,3337.6


In [6]:
data_dict = fl.load(dest_dir / f"{tstamp}_python_data.h5")
numerical = ["quantita", "imp. unit."]
budgets_tot, budgets_dec = data_dict["budgets"], data_dict["budgets_dec"]

budgets_tot = budgets_tot.set_index(["commessa", "codice", "fase"]).drop(["costo u.", "imp.comp."], axis=1)
budgets_dec = budgets_dec.set_index(["commessa", "codice", "fase"]).drop(["costo u.", "imp.comp."], axis=1)

dec_al, tot_al = budgets_dec.align(budgets_tot)
dec_al.loc[:, numerical] = dec_al.loc[:, numerical].fillna(0)
tot_al.loc[:, numerical] = tot_al.loc[:, numerical].fillna(0)

deltas = tot_al["quantita"] - dec_al["quantita"]
deltas.to_excel(dest_dir / f"{tstamp}_deltas.xlsx")
deltas[deltas < 0].to_excel(dest_dir / f"{tstamp}_negative-deltas.xlsx")


print("Before dropping weird negatives: ", (tot_al["quantita"] - dec_al["quantita"]).sum())

problematic = dec_al[(dec_al["quantita"] > tot_al["quantita"]) & (tot_al["quantita"] > 0)].index

Before dropping weird negatives:  14974437.870095383


In [ ]:
tot_al.loc[problematic, :]

In [ ]:


dec_al = dec_al.drop(problematic)
tot_al = tot_al.drop(problematic)

print("before dropping all negatives: ", (tot_al["quantita"] - dec_al["quantita"]).sum())
negative_correct = dec_al[(dec_al["quantita"] > tot_al["quantita"])].index

dec_al = dec_al.drop(negative_correct)
tot_al = tot_al.drop(negative_correct)

print("dropping all negatives: ", (tot_al["quantita"] - dec_al["quantita"]).sum())

In [ ]:
dec_al, tot_al = budgets_dec.align(budgets_tot)
dec_al.loc[:, numerical] = dec_al.loc[:, numerical].fillna(0)
tot_al.loc[:, numerical] = tot_al.loc[:, numerical].fillna(0)

In [ ]:
problematic = dec_al[(dec_al["quantita"] > tot_al["quantita"]) & (tot_al["quantita"] > 0)].index

In [ ]:
dec_al.loc[problematic, :]

In [ ]:
tot_al.loc[problematic, :]

In [ ]:
dec_al, tot_al = budgets_dec.align(budgets_tot)
all_diffs = dec_al["quantita"] - tot_al["quantita"]

not_matching_2021 = set(budgets_dec.index) - set(budgets_tot.index)
negative_diffs = set(all_diffs[all_diffs < 0].index)
budgets_dec.drop(list(negative_diffs - not_matching_2021))

dec_al, tot_al = budgets_dec.align(budgets_tot)
numerical = ["quantita"]
dec_al.loc[:, numerical] = dec_al.loc[:, numerical].fillna(0)
tot_al.loc[:, numerical] = tot_al.loc[:, numerical].fillna(0)

not_matching_2021 = set(budgets_dec.index) - set(budgets_tot.index)
negative_diffs = set(all_diffs[all_diffs < 0].index)

tot_al = tot_al.drop(list(negative_diffs))
dec_al = dec_al.drop(list(negative_diffs))

In [ ]:
(tot_al["quantita"] - dec_al["quantita"]).sum()

In [ ]:
(tot_al["quantita"] - dec_al["quantita"]).sum()

In [ ]:
all_diffs = dec_al - 

In [ ]:
s = budgets_dec.loc[list(negative_diffs - not_matching_2021)]

In [ ]:
print("December 2021")
budgets_dec.drop(list(negative_diffs - not_matching_2021))

In [ ]:
print("Sum 2022")
budgets_tot.loc[list(negative_diffs - not_matching_2021)].head()

In [ ]:
all_diffs = tot_al[numerical] - dec_al[numerical]

In [ ]:
all_diffs["quantita"].sum()

In [ ]:
 17930866.01